<a href="https://colab.research.google.com/github/SATHYARANGARAJ53/medicine_inventory/blob/main/projectmodel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.tsa.seasonal import seasonal_decompose
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from prophet import Prophet

# Load dataset
df = pd.read_csv("sample_Au24.xlsx.csv")

# Generate synthetic stock data
df['Stock Level'] = np.random.randint(50, 500, df.shape[0])
df['Weekly Consumption'] = np.random.randint(10, 50, df.shape[0])
df['Stock Expiry (Days)'] = np.random.randint(10, 120, df.shape[0])

# CHAID for real-time stock analysis
contingency_table = pd.crosstab(df['Stock Expiry (Days)'] < 30, df['Stock Level'] < 100)
chi2, p, _, _ = chi2_contingency(contingency_table)
print(f"CHAID Test - Chi2: {chi2}, P-Value: {p}")

# LSTM for utilization trends
sequence_length = 5
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df['Weekly Consumption'].iloc[i:i + sequence_length].values)
    y.append(df['Weekly Consumption'].iloc[i + sequence_length])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, verbose=1)

# Prophet for demand forecasting
prophet_df = df[['Medicine Name', 'Weekly Consumption']].rename(columns={'Weekly Consumption': 'y'})
prophet_df['ds'] = pd.date_range(start='2025-01-01', periods=len(df), freq='W')
prophet_model = Prophet()
prophet_model.fit(prophet_df)
future = prophet_model.make_future_dataframe(periods=12, freq='W')
forecast = prophet_model.predict(future)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

CHAID Test - Chi2: 0.10147455208498499, P-Value: 0.7500672385539535


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 790.0540
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 619.1998
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 322.8758
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 237.9004
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 263.4074
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 206.0648
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 203.5619
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 217.4628
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 170.6277
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 173.1883


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp9e66i4g/pgpkmskq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp9e66i4g/ff7nebfn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=91055', 'data', 'file=/tmp/tmpp9e66i4g/pgpkmskq.json', 'init=/tmp/tmpp9e66i4g/ff7nebfn.json', 'output', 'file=/tmp/tmpp9e66i4g/prophet_modelmq9u019b/prophet_model-20250131100332.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:03:32 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:03:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


            ds       yhat  yhat_lower  yhat_upper
127 2027-06-13  25.361611   11.635586   38.921493
128 2027-06-20  31.137034   17.838029   45.054421
129 2027-06-27  37.617878   23.391087   51.189323
130 2027-07-04  39.205126   25.848036   53.853248
131 2027-07-11  34.125007   19.981724   48.355087
